In [1]:
# inspired by https://github.com/ritvikmath/StarbucksStoreScraping/blob/master/Scraper.ipynb

In [2]:
import requests
import re
import pandas as pd
import numpy as np
from pandas import DataFrame
import json

In [3]:
url = "https://www.zipcodestogo.com/Saint%20louis/MO/"
tables = pd.read_html(url,skiprows=1)
stl_zip = tables[0]
stl_zip.columns = [stl_zip.iloc[0]]
stl_zip.drop(index=0, inplace=True)
stl_zip.reset_index(inplace=True, drop=True)
stl_zip_df = np.array(stl_zip)
stl_zip_list = pd.DataFrame(stl_zip_df).iloc[:,:1]
stl_zip_list.columns = ['zip']
stl_zip_list = list(stl_zip_list['zip'])
stl_zip_list

['63001',
 '63005',
 '63006',
 '63011',
 '63017',
 '63021',
 '63022',
 '63024',
 '63025',
 '63026',
 '63031',
 '63032',
 '63033',
 '63034',
 '63038',
 '63040',
 '63042',
 '63043',
 '63044',
 '63045',
 '63074',
 '63088',
 '63099',
 '63105',
 '63114',
 '63117',
 '63119',
 '63121',
 '63122',
 '63123',
 '63124',
 '63125',
 '63126',
 '63127',
 '63128',
 '63129',
 '63130',
 '63131',
 '63132',
 '63133',
 '63134',
 '63135',
 '63136',
 '63137',
 '63138',
 '63140',
 '63141',
 '63143',
 '63144',
 '63145',
 '63146',
 '63151',
 '63167',
 '63198']

In [4]:
#save zip code list for other uses
with open('../zip_list.txt', 'w') as f:
    f.write('\n'.join(stl_zip_list))

In [5]:
def processResponse(r):
    #parse out each store's info
    stores = re.findall(r'"storeNumber":.*?"slug"', r)
    storeInfo = []
    for store in stores:
        #parse out info about each store
        storeInfo.append(list(re.findall(r'"storeNumber":"(.*?)".*?"name":"(.*?)".*?"latitude":(.*?),.*?"longitude":(.*?)}.*?"city":"(.*?)".*?"countrySubdivisionCode":"(.*?)".*?"postalCode":"(.*?)"', store)[0]))
    return storeInfo

In [6]:
allStores = []
for idx,z in enumerate(stl_zip_list):
    if idx%10 == 0:
        print(idx,'/',len(stl_zip_list))
    #search for 100 stores centered in given zip
    r = requests.get('https://www.starbucks.com/store-locator?map=34.216464,-118.656046,11z&place='+z)
    if r.status_code != 200:
        raise SystemExit
    storeInfoList = processResponse(r.text)
    for storeInfo in storeInfoList:
        storeInfo[6] = storeInfo[6][:5]
    allStores += storeInfoList

0 / 54
10 / 54
20 / 54
30 / 54
40 / 54
50 / 54


In [7]:
#construct non-duplicated list of stores
seenStoreIds = []
stlStores = []
for store in allStores:
    if store[0] in seenStoreIds:
        continue
    else:
        stlStores.append(store)
        seenStoreIds.append(store[0])

In [8]:
dfSbux = pd.DataFrame(columns=['id', 'strLocation', 'latitude', 'longitude', 'city', 'state', 'zip'])
for i,col in enumerate(dfSbux.columns):
    dfSbux[col] = [item[i] for item in stlStores]

In [9]:
dfSbux.latitude = dfSbux.latitude.apply(lambda x: float(x))
dfSbux.longitude = dfSbux.longitude.apply(lambda x: float(x))

In [10]:
dfSbuxMO = dfSbux.loc[dfSbux['state'] == 'MO']
dfSbuxIL = dfSbux.loc[dfSbux['state'] == 'IL']
dfSbuxConcat = [dfSbuxMO, dfSbuxIL]
dfSbuxFinal = pd.concat(dfSbuxConcat)
dfSbuxFinal.reset_index(drop=True, inplace=True)
dfSbuxFinal["Store"] = "Starbucks"

In [11]:
dfSbuxFinal.to_csv('../sbux_loc.csv', index = False)
check = pd.read_csv('../sbux_loc.csv')
check

,id,strLocation,latitude,longitude,city,state,zip,Store
0,13473-107465,32nd & Range Line,37.052427,-94.478940,Joplin,MO,64804,Starbucks
1,76533-100210,Target Joplin T-774,37.085213,-94.473855,Joplin,MO,64801,Starbucks
2,9566-96850,3rd & Range Line,37.088028,-94.477036,Joplin,MO,64801,Starbucks
3,22191-212244,St. Louis Premium Outlets,38.644100,-90.631922,Chesterfield,MO,63005,Starbucks
4,7994-95384,Taylor & Manchester,38.582168,-90.628030,Wildwood,MO,63040,Starbucks
...,...,...,...,...,...,...,...,...
84,49008-266131,157 & Center Grove Rd,38.787750,-89.978912,Edwardsville,IL,62025,Starbucks
85,13852-108653,Troy & Governor's Pkwy,38.788790,-89.951888,Edwardsville,IL,62025,Starbucks
86,56240-288602,TARGET EDWARDSVILLE T-733,38.782621,-89.949238,Edwardsville,IL,62025,Starbucks
87,76860-116533,Target Belleville T-2330,38.511200,-90.014600,Belleville,IL,62226,Starbucks
